In [ ]:
!pip install praw
!pip install autocorrect
!pip install vaderSentiment
!pip install feel_it
!pip install pywaffle
!pip install afinn
!pip install pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 974.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.8/622.8 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622364 sha256=cf51c08eb00618ac83cff8684fb14d4a776eee0fa5cc2a310b3011a9960afddb
  Stored in directory: /root/.cache/pip/wheels/b5/7b/6d/b76b29ce11ff8e2521c8c7dd0e5bfee4fb1789d76193124343
Successfully built autocorrect
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.6/25.6 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 814.6 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for afinn: filename=afinn-0.1-py3-none-any.whl size=53429 sha256=d7b144be1ef3bb4931acfff61321c907971a67b71f1f6007321c2fdc47cc570c
  Stored in directory: /root/.cache/pip/wheels/b0/05/90/43f791

In [ ]:
pip install --upgrade pyLDAvis gensim

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
import praw
import requests
import pandas as pd
import json
import re
from datetime import datetime
from autocorrect import Speller
from nltk.stem import WordNetLemmatizer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from feel_it import EmotionClassifier
from pywaffle import Waffle
import matplotlib.pyplot as plt
from pandas.plotting import parallel_coordinates
from matplotlib.colors import ListedColormap
import seaborn as sns
import nltk
import string
from collections import Counter
import itertools
from afinn import Afinn
from gensim import corpora
import networkx
from gensim.models.phrases import Phraser
from gensim.models import Phrases
import gensim.models as models
from gensim.models.ldamulticore import LdaMulticore, LdaModel
from gensim.models import CoherenceModel
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open("/content/drive/MyDrive/SMA/RedditCredentials.json", "r") as f:
    credentials = json.load(f)

In [ ]:
reddit = praw.Reddit(client_id=credentials['client_id'],
                     client_secret=credentials['client_secret'],
                     password=credentials['password'],
                     user_agent='Tutorial by /user/marcosallustio',
                     username=credentials['username'],
                     check_for_async=False)

I extract the comments for each post in the subreddit and create a dataframe where I have for each row a comment and other associated information


In [ ]:
import time

comments_data = []

for submission in reddit.subreddit("marvelstudios").search('avengers OR endgame', limit=100):
    submission_data = {
        'author_submission': submission.author.name if submission.author else None,
        'title_post': submission.title,
        'score_post': submission.score,
        'url_post': submission.url,
        'created_utc_post': submission.created_utc,
    }

    # Extract comments
    submission.comments.replace_more(limit=30)
    for comment in submission.comments.list():
        comment_data = {
            'comment_id': comment.id,
            'author_comment': comment.author.name if comment.author else None,
            'body': comment.body,
            'created_utc_comment': comment.created_utc,
            'score_comment': comment.score
        }
        comments_data.append({**submission_data, **comment_data})


    time.sleep(2)


# Create DataFrame
df = pd.DataFrame(comments_data)

In [ ]:
df

,author_submission,title_post,score_post,url_post,created_utc_post,comment_id,author_comment,body,created_utc_comment,score_comment
0,HazzaPD,Avengers: Endgame Official Trailer,49240,https://twitter.com/russo_brothers/status/1106...,1.552565e+09,eii48xr,Reverse-I_am_Organic,This trailer came outta left field. I’m so fu...,1.552565e+09,4425
1,HazzaPD,Avengers: Endgame Official Trailer,49240,https://twitter.com/russo_brothers/status/1106...,1.552565e+09,eii6utj,BIGDOOKY15,"3 HOUR RUNTIME. \n\nDO IT, YOU COWARDS.",1.552567e+09,3450
2,HazzaPD,Avengers: Endgame Official Trailer,49240,https://twitter.com/russo_brothers/status/1106...,1.552565e+09,eii4rjt,None,WE'RE LIKE SIX WEEKS AWAY. KEEP HOLDING ON,1.552565e+09,5397
3,HazzaPD,Avengers: Endgame Official Trailer,49240,https://twitter.com/russo_brothers/status/1106...,1.552565e+09,eii5cyv,Sigma1977,Rocket riding on War Machine's back like he di...,1.552566e+09,6609
4,HazzaPD,Avengers: Endgame Official Trailer,49240,https://twitter.com/russo_brothers/status/1106...,1.552565e+09,eii4w7w,None,So much for the people saying the suits won't ...,1.552566e+09,1176
...,...,...,...,...,...,...,...,...,...,...
112562,PhoOhThree,Avengers: Endgame - Red Carpet World Premiere ...,926,https://www.reddit.com/r/marvelstudios/comment...,1.555958e+09,eljr44s,LeMoineSpectre,I'd hardly call that sexual harassment. He did...,1.555991e+09,2
112563,PhoOhThree,Avengers: Endgame - Red Carpet World Premiere ...,926,https://www.reddit.com/r/marvelstudios/comment...,1.555958e+09,eljr7rp,-OrangeLightning4,You found the correct way to respond to trolls.,1.555991e+09,5
112564,PhoOhThree,Avengers: Endgame - Red Carpet World Premiere ...,926,https://www.reddit.com/r/marvelstudios/comment...,1.555958e+09,eljsbjz,Funbun71,I love you,1.555992e+09,3
112565,PhoOhThree,Avengers: Endgame - Red Carpet World Premiere ...,926,https://www.reddit.com/r/marvelstudios/comment...,1.555958e+09,eljrs9h,None,He’s the chosen one,1.555992e+09,2


In [ ]:
df['created_utc_comment'] = df['created_utc_comment'].apply(lambda x: datetime.fromtimestamp(x))

In [ ]:
df['created_utc_comment']

0        2019-03-14 12:01:20
1        2019-03-14 12:40:24
2        2019-03-14 12:09:54
3        2019-03-14 12:19:08
4        2019-03-14 12:12:04
                 ...        
112562   2019-04-23 03:42:40
112563   2019-04-23 03:44:06
112564   2019-04-23 04:00:29
112565   2019-04-23 03:52:42
112566   2019-04-23 04:06:46
Name: created_utc_comment, Length: 112567, dtype: datetime64[ns]

In [ ]:
df= df.sort_values(by=['created_utc_comment']).reset_index(drop=True)

In [ ]:
df

,author_submission,title_post,score_post,url_post,created_utc_post,comment_id,author_comment,body,created_utc_comment,score_comment
0,mglorbrto87,Avengers Endgame runtime still roughly at 3 ho...,4089,http://collider.com/avengers-endgame-runtime-u...,1.549412e+09,efu1uhg,iAMA_Leb_AMA,Its insane how this film could be 3 hours long...,2019-02-06 00:22:21,1804
1,mglorbrto87,Avengers Endgame runtime still roughly at 3 ho...,4089,http://collider.com/avengers-endgame-runtime-u...,1.549412e+09,efu20ii,mutdude12,Can't even imagine being part of the test audi...,2019-02-06 00:24:37,733
2,mglorbrto87,Avengers Endgame runtime still roughly at 3 ho...,4089,http://collider.com/avengers-endgame-runtime-u...,1.549412e+09,efu23r9,PeterSleepsInaParker,Dude we only can dream!,2019-02-06 00:25:50,194
3,mglorbrto87,Avengers Endgame runtime still roughly at 3 ho...,4089,http://collider.com/avengers-endgame-runtime-u...,1.549412e+09,efu247d,STALAL,REALLY wish they give it the sendoff it deserv...,2019-02-06 00:26:00,326
4,mglorbrto87,Avengers Endgame runtime still roughly at 3 ho...,4089,http://collider.com/avengers-endgame-runtime-u...,1.549412e+09,efu2ihn,None,"Pls keep it at 3 hours, or just release an ext...",2019-02-06 00:31:18,292
...,...,...,...,...,...,...,...,...,...,...
112562,MonsieurA,"October 3, 2023: today is Day 1 in the Avenger...",5750,https://i.redd.it/tcsuyihnexrb1.jpg,1.696312e+09,k3lbmeh,pipelowarrior,Thanks,2023-10-05 16:10:56,1
112563,MonsieurA,"October 3, 2023: today is Day 1 in the Avenger...",5750,https://i.redd.it/tcsuyihnexrb1.jpg,1.696312e+09,k3ldnbf,Caleb_Murphy,"I'm serious, man. I'm not trying to put you do...",2023-10-05 16:23:00,3
112564,MonsieurA,"October 3, 2023: today is Day 1 in the Avenger...",5750,https://i.redd.it/tcsuyihnexrb1.jpg,1.696312e+09,k3le06r,pipelowarrior,"Ok, I will\n\nBut really\nThank you\n\nIm not ...",2023-10-05 16:25:05,3
112565,MonsieurA,"October 3, 2023: today is Day 1 in the Avenger...",5750,https://i.redd.it/tcsuyihnexrb1.jpg,1.696312e+09,k3md959,WeatherBois,A party that ends when somebody’s heads on a s...,2023-10-05 19:52:09,1


I filter the dataframe starting from the release date of the film for the duration of one year


In [ ]:
df = df[(df['created_utc_comment'] >= datetime(2019, 4, 26, 0, 0)) & (df['created_utc_comment'] <= datetime(2020, 4, 26, 23, 59, 59))]

In [ ]:
df

,author_submission,title_post,score_post,url_post,created_utc_post,comment_id,author_comment,body,created_utc_comment,score_comment
40949,JonLuca,The Official AVENGERS: ENDGAME Release Megathr...,2287,https://www.reddit.com/r/marvelstudios/comment...,1.556226e+09,elsgggh,millionsofgoats,I’m a bit upset about how they handled Nat’s d...,2019-04-26 00:00:06,12
40950,ScottFromScotland,The Official AVENGERS: ENDGAME Release Megathr...,1382,https://www.reddit.com/r/marvelstudios/comment...,1.556197e+09,elsggze,tkv_is_pro,"So basically, the DragonballZ type of time tra...",2019-04-26 00:00:17,2
40951,JonLuca,The Official AVENGERS: ENDGAME Release Megathr...,2287,https://www.reddit.com/r/marvelstudios/comment...,1.556226e+09,elsghb0,WonkDog,Nanotech brah. Tony is peak intelligence. Even...,2019-04-26 00:00:23,10
40952,JonLuca,The Official AVENGERS: ENDGAME Release Megathr...,2287,https://www.reddit.com/r/marvelstudios/comment...,1.556226e+09,elsgh9k,anotherboreddude,Honestly this felt like such an ending that I ...,2019-04-26 00:00:23,9
40953,JonLuca,The Official AVENGERS: ENDGAME Release Megathr...,2287,https://www.reddit.com/r/marvelstudios/comment...,1.556226e+09,elsgi3q,OnceOnThisIsland,"He's there, but they probably don't speak.",2019-04-26 00:00:40,5
...,...,...,...,...,...,...,...,...,...,...
108708,mattwart,My Captain America colour pencil drawing (Aven...,18218,https://i.redd.it/6widbbiqd3j41.jpg,1.582646e+09,fiuwpnr,mattwart,"Yup, the drawing was done with colour pencil (...",2020-02-26 20:40:53,2
108709,gianben123,Avengers: Endgame | Tony & teenage Morgan Star...,17416,https://v.redd.it/5hhvq02eh8y31,1.573555e+09,fkkphdx,choochoo789,God yes,2020-03-15 16:19:50,2
108710,UnrealLuigi,'Avengers: Endgame' limited edition poster by ...,18043,https://i.redd.it/2cmyvbdx52h41.jpg,1.581760e+09,fmh8f03,blackbutterfree,"Looks to be missing Howard the Duck, Kraglin (...",2020-04-05 07:03:57,1
108711,gianben123,Avengers: Endgame | Tony & teenage Morgan Star...,17416,https://v.redd.it/5hhvq02eh8y31,1.573555e+09,fojdf5j,hwengiue,"First of all, Katherine Langford has nothing t...",2020-04-25 12:26:16,2


In [ ]:
df['author_submission'].unique()

array(['JonLuca', 'ScottFromScotland', 'KostisPat257', 'Flamma_Man',
       'MarvelStudiosMod', 'jesjes3000', 'HazzaPD', 'PhoOhThree',
       'quickfund', 'dannys717', 'sekans17', 'The_Asian_Hamster',
       'mrnicegy26', 'Nort8n', 'najmussajid', 'SamanthasCosplay',
       'gianben123', 'Sisiwakanamaru', 'mglorbrto87', 'Vexse_',
       'xThreatLevelMidnight', 'Whyyygsus', 'VanBeFresk',
       'ThoseAreBoldWordsB', 'matthewbassit', 'kahlkorver', 'lochnussy',
       'chanma50', 'JakeM917', 'LordHyperBreath', 'NaiadoftheSea',
       'MarvelKenneth', 'da0ur', 'oystermilk', 'RonnieMaz',
       'Griffdogg123', 'YassPanda25', 'ENusatron', 'Russo_Brothers',
       'Kazrules', 'Tristan_Culbert', 'MangoJam18', 'aaronp613',
       'qwertypop123455', 'Eric191', 'itsevilR', 'UnrealLuigi',
       'mattwart'], dtype=object)

In [ ]:
df.to_csv('avengers.csv',index=False)